## MAIN DATA PROCESSING
- The notebook is used to create and aggregate the necessary data for the empirical analysis notebook

In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import random

pd.set_option("display.max_rows", 100)

In [2]:
fh_returns = "/Users/henrycosentino/Desktop/Python/Projects/Event Drift/Data Processing/return_data.xlsx"
fh_earnings_dates = "/Users/henrycosentino/Desktop/Python/Projects/Event Drift/Data Processing/earnings_dates_data.xlsx"

returns_df = pd.read_excel(fh_returns)
earnings_dates_df = pd.read_excel(fh_earnings_dates, sheet_name='Values 2.0')

In [3]:
earnings_dates_df.head()

,PODD US EQUITY,Unnamed: 1,CRWD US EQUITY,Unnamed: 3,NWSA US EQUITY,Unnamed: 5,SBAC US EQUITY,Unnamed: 7,UPS US EQUITY,Unnamed: 9,...,PYPL US EQUITY,Unnamed: 1031,WHR US EQUITY,Unnamed: 1033,GM US EQUITY,Unnamed: 1035,NI US EQUITY,Unnamed: 1037,LW US EQUITY,Unnamed: 1039
0,2025-02-20,NaN,2025-03-04,2025:A,2025-02-05,2025:C2,2025-04-28,NaN,2025-04-29,NaN,...,2025-04-29,2025:C1,2025-04-23,2025:C1,2025-04-29,2025:C1,2025-02-12,2024:A,2025-04-03,2025:C3
1,2025-02-20,2024:Q4,2025-03-04,2025:Q4,2025-02-05,2025:Q2,2025-04-28,2025:Q1,2025-04-29,NaN,...,2025-04-29,2025:Q1,2025-04-23,2025:Q1,2025-04-29,2025:Q1,2025-02-12,2024:C4,2025-04-03,2025:Q3
2,2024-11-07,2024:Q3,2024-11-26,2025:Q3,2024-11-07,2025:C1,2025-02-24,2024:A,2025-01-30,NaN,...,2025-02-04,2024:A,2025-01-29,2024:A,2025-01-28,2024:A,2025-02-12,2024:Q4,2024-12-19,2025:C2
3,2024-08-08,2024:Q2,2024-08-28,2025:Q2,2024-11-07,2025:Q1,2025-02-24,2024:C4,2025-01-30,NaN,...,2025-02-04,2024:C4,2025-01-29,2024:C4,2025-01-28,2024:C4,2024-10-30,2024:C3,2024-12-19,2025:Q2
4,2024-05-09,2024:Q1,2024-06-04,2025:Q1,2024-08-08,2024:A,2025-02-24,2024:Q4,2025-01-30,NaN,...,2025-02-04,2024:Q4,2025-01-29,2024:Q4,2025-01-28,2024:Q4,2024-10-30,2024:Q3,2024-10-01,2025:C1


In [4]:
# Cleaning up earnings dataframe and transforming into a dictionary

cols_to_keep = [col for col in earnings_dates_df.columns if col.endswith('EQUITY')]
cols_to_keep.remove('SPY US EQUITY') # Remove SPY since S&P 500 isn't used in the earnings day variable
earnings_dates_df = earnings_dates_df[cols_to_keep]
earnings_dates_df = earnings_dates_df.iloc[1:].reset_index(drop=True)

for col in earnings_dates_df.columns:
    earnings_dates_df[col] = pd.to_datetime(earnings_dates_df[col]).dt.strftime('%Y-%m-%d')

earnings_dates_dict = earnings_dates_df.to_dict(orient='list')

In [5]:
# Creating a unique earnings date dictionary

earnings_dates_dict_unique = {}

for k, v in earnings_dates_dict.items():
    v = set(v)
    if np.nan in v:
        v.remove(np.nan)
    if "NaN" in v:
        v.remove("NaN")
    pos = k.find(" ")
    k = k[:pos]
    earnings_dates_dict_unique[k] = v

## Main Dataframe Construction

In [6]:
returns_df = returns_df.set_index('Date')
equity_ls = list(returns_df.columns)
equity_ls.remove('SPY')
returns_df.head()

,PODD,CRWD,NWSA,SBAC,UPS,VZ,IPGP,PFG,NXPI,DE,...,AIG,BIO,HES,MSI,MDT,PYPL,WHR,GM,NI,LW
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,-0.008353,-0.008622,0.021216,-0.008797,-0.002307,-0.005700,0.010765,0.013273,0.010058,0.020778,...,0.008377,0.005756,0.013621,0.003041,0.009784,0.023851,0.006236,0.021312,-0.023707,-0.022550
2020-01-03,0.010072,0.026497,-0.002770,0.009043,-0.000599,-0.010647,-0.031813,-0.014714,-0.009647,-0.007407,...,-0.007728,-0.014456,0.031305,0.017014,-0.005936,-0.017968,-0.017042,-0.028357,0.008094,0.112618
2020-01-06,0.036914,0.085911,-0.002778,0.000913,-0.004455,-0.002152,0.010224,0.001639,-0.007070,0.007177,...,0.000779,0.014314,0.014605,0.015452,0.008869,0.012964,0.009389,-0.013216,-0.000365,-0.030569
2020-01-07,-0.011979,-0.006532,0.009749,-0.015751,-0.001721,-0.011117,0.005793,0.006363,0.012975,-0.017363,...,-0.005642,0.022875,0.001129,-0.007189,-0.003482,-0.004538,0.000339,-0.019252,0.008762,0.005623
2020-01-08,0.007229,0.016621,0.002758,0.010865,0.005690,0.001846,-0.000347,0.004336,0.002499,0.012432,...,0.011739,0.003285,-0.034113,0.001026,0.017556,0.019604,0.013981,-0.014224,0.001086,0.000548


In [7]:
# Creating rolling average and standard deviation columns

rolling_stats_dict = {}
for col in returns_df.columns:
    roll_avg_col = f"{col}_rollAvg"
    roll_stdev_col = f"{col}_rollStdev"
    rolling_stats_dict[roll_avg_col] = returns_df[col].rolling(window=30).mean()
    rolling_stats_dict[roll_stdev_col] = returns_df[col].rolling(window=30).std()

rolling_df = pd.DataFrame(rolling_stats_dict, index=returns_df.index)
returns_df = pd.concat([returns_df, rolling_df], axis=1)

In [8]:
# Joining Returns w/ All Days Between start of 2020 and end of 2024 

start = "2022-01-01"
end = "2024-12-31"
date_index = pd.date_range(start=start, end=end, freq='D')
dates_df = pd.DataFrame(index=date_index)
dates_df.index.name = 'Date'
main_df = dates_df.join(returns_df, how='left')
main_df.head()

,PODD,CRWD,NWSA,SBAC,UPS,VZ,IPGP,PFG,NXPI,DE,...,PYPL_rollAvg,PYPL_rollStdev,WHR_rollAvg,WHR_rollStdev,GM_rollAvg,GM_rollStdev,NI_rollAvg,NI_rollStdev,LW_rollAvg,LW_rollStdev
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,0.036194,-0.031355,0.020619,-0.023521,-0.003919,0.009238,0.009701,0.012167,0.015102,0.021056,...,-0.000729,0.020829,0.000578,0.016497,-0.000240,0.028249,0.003100,0.010507,0.004094,0.020724
2022-01-04,-0.007798,-0.046085,-0.007905,-0.027404,0.017752,0.019641,0.004315,0.032919,0.007136,0.060495,...,-0.000233,0.020167,0.001595,0.017291,0.002534,0.031327,0.003088,0.010506,0.004081,0.020732
2022-01-05,-0.066167,-0.052328,-0.033643,-0.050587,-0.012564,0.010286,-0.029102,-0.018514,-0.035642,0.002047,...,-0.000216,0.020149,0.000868,0.017564,-0.000206,0.031839,0.002939,0.010437,0.003407,0.020613


In [9]:
# Creation of dummy variables for earnings releases 

dummy_cols_dict = {}

for col in equity_ls:
    if col in earnings_dates_dict_unique:
        earnings_dates = earnings_dates_dict_unique[col]
        col_name = f"{col}_earnDateDummy"
        dummys = []
        for day in main_df.index:
            day_str = day.strftime('%Y-%m-%d') # Convert index datetime to string for comparison
            if day_str in earnings_dates:
                dummys.append(1)
            else:
                dummys.append(0)
        dummy_cols_dict[col_name] = dummys
    else:
        print(f"Warning: Ticker '{col}' not found in earnings date dictionary. Skipping dummy creation.")

# Add a second dummy after each earnings day to minimize getting a incorrect signal (autocorrelation, volatility follows volatility)
for k, v in dummy_cols_dict.items():
    step = 0
    while step < (len(v) - 1):
        if v[step] == 1 and v[step+1] == 0:
            v[step+1] = 1
            step += 2
        else:
            step += 1

earningsDummiesCols_ls = list(dummy_cols_dict.keys())
earningsDummies_df = pd.DataFrame(dummy_cols_dict, index=main_df.index)
main_df = pd.concat([main_df, earningsDummies_df], axis=1)

In [10]:
# Our main dataframe so far... we need to identify events next!

main_df.head()

,PODD,CRWD,NWSA,SBAC,UPS,VZ,IPGP,PFG,NXPI,DE,...,AIG_earnDateDummy,BIO_earnDateDummy,HES_earnDateDummy,MSI_earnDateDummy,MDT_earnDateDummy,PYPL_earnDateDummy,WHR_earnDateDummy,GM_earnDateDummy,NI_earnDateDummy,LW_earnDateDummy
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2022-01-03,0.036194,-0.031355,0.020619,-0.023521,-0.003919,0.009238,0.009701,0.012167,0.015102,0.021056,...,0,0,0,0,0,0,0,0,0,0
2022-01-04,-0.007798,-0.046085,-0.007905,-0.027404,0.017752,0.019641,0.004315,0.032919,0.007136,0.060495,...,0,0,0,0,0,0,0,0,0,0
2022-01-05,-0.066167,-0.052328,-0.033643,-0.050587,-0.012564,0.010286,-0.029102,-0.018514,-0.035642,0.002047,...,0,0,0,0,0,0,0,0,0,0


## Event Identification

In [11]:
# Function for changing event identification

def id_event(df, equity_ls, sigma_lvl_mrkt=1, sigma_lvl_equity=4, key="_potEvent"):

    event_dict = {}

    for col in equity_ls:
        # Condition One: Not an earnings day
        cond1 = df[f"{col}_earnDateDummy"] < 1

        # Condition Two: S&P 500 volatility range
        lower_bound_market = df['SPY_rollAvg'] - (df['SPY_rollStdev'] * sigma_lvl_mrkt)
        upper_bound_market = df['SPY_rollAvg'] + (df['SPY_rollStdev'] * sigma_lvl_mrkt)
        cond2 = (df['SPY'] >= lower_bound_market) & (df['SPY'] <= upper_bound_market)

        # Condition Three: Chosen equity volatility range
        if key == "_potEvent":
            lower_bound_equity = df[f"{col}_rollAvg"] - (df[f"{col}_rollStdev"] * sigma_lvl_equity)
            upper_bound_equity = df[f"{col}_rollAvg"] + (df[f"{col}_rollStdev"] * sigma_lvl_equity)
            cond3 = (df[col] < lower_bound_equity) | (df[col] > upper_bound_equity)
        else:
            lower_bound_equity = df[f"{col}_rollAvg"] - (df[f"{col}_rollStdev"] * sigma_lvl_equity)
            upper_bound_equity = df[f"{col}_rollAvg"] + (df[f"{col}_rollStdev"] * sigma_lvl_equity)
            cond3 = (df[col] > lower_bound_equity) & (df[col] < upper_bound_equity)

        # Identify the days where these conditions hold
        result = np.where(cond1 & cond2 & cond3, 1, 0)
   
        event_dict[col+key] = result

    # Return a dictionary where these results hold true
    return event_dict

In [12]:
# Creating event dummy columns and adding to the main data frame

potEvent_dict = id_event(main_df, equity_ls)
eventCols_ls = list(potEvent_dict.keys())

eventDummies_df = pd.DataFrame(potEvent_dict, index=main_df.index)
event_df = pd.concat([main_df, eventDummies_df], axis=1)
event_df.head()

,PODD,CRWD,NWSA,SBAC,UPS,VZ,IPGP,PFG,NXPI,DE,...,AIG_potEvent,BIO_potEvent,HES_potEvent,MSI_potEvent,MDT_potEvent,PYPL_potEvent,WHR_potEvent,GM_potEvent,NI_potEvent,LW_potEvent
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2022-01-03,0.036194,-0.031355,0.020619,-0.023521,-0.003919,0.009238,0.009701,0.012167,0.015102,0.021056,...,0,0,0,0,0,0,0,0,0,0
2022-01-04,-0.007798,-0.046085,-0.007905,-0.027404,0.017752,0.019641,0.004315,0.032919,0.007136,0.060495,...,0,0,0,0,0,0,0,0,0,0
2022-01-05,-0.066167,-0.052328,-0.033643,-0.050587,-0.012564,0.010286,-0.029102,-0.018514,-0.035642,0.002047,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Get dates where potential events occur

truncated_event_df = event_df.iloc[:1000].copy() # Truncate to only capture event dates with a maximum of 60 trading days left in 2024

eventDate_dict = {}
for col in equity_ls:
    sub_df = truncated_event_df[[col, f'{col}_potEvent']][truncated_event_df[f'{col}_potEvent'] == 1]
    eventDate_dict[col] = list(sub_df.index)

In [14]:
# Create set of potential event dates

eventDate_set = set()
for k, v in eventDate_dict.items():
    if len(v) == 0:
        continue
    else:
        for date in v:
            date_str = date.strftime('%Y-%m-%d')
            ret = returns_df[k].loc[date_str]
            eventDate_set.add((k, date_str, ret))

In [15]:
## EVENTS ##

# Randomly sampling the equities with events

random.seed(42)
random_equityEvent_ls = random.sample(sorted(list(eventDate_set)), 55) # Randomly sample n = 50 (extra 5 incase incorrect signal), based on our quantitative signal for finding a potential corporate event
random_equityEvent_ls

[('WDC', '2022-05-03', 0.1446587314899124),
 ('DVA', '2023-10-11', -0.1686020989862133),
 ('ALL', '2022-10-20', -0.1290418222547864),
 ('ILMN', '2022-01-11', 0.1698134137685012),
 ('HRL', '2023-10-12', -0.09826134245621898),
 ('HBAN', '2023-03-13', -0.1682871678509755),
 ('FANG', '2024-02-12', 0.0938447572511345),
 ('DD', '2024-01-24', -0.1404472580640097),
 ('STZ', '2023-07-19', 0.05347395480212302),
 ('CI', '2023-12-11', 0.1668082713857855),
 ('ULTA', '2024-04-03', -0.1533635753015731),
 ('MRK', '2024-03-27', 0.04963357815695479),
 ('ANSS', '2023-12-22', 0.1808286273321169),
 ('XEL', '2024-02-29', -0.08635346821231638),
 ('UNP', '2023-02-27', 0.1009233177720648),
 ('GLW', '2024-07-08', 0.119927059573911),
 ('HII', '2023-10-09', 0.09342286907370778),
 ('PFE', '2023-05-22', 0.05384834509326142),
 ('TTWO', '2022-01-10', -0.1312879696782101),
 ('GD', '2023-10-09', 0.08429566604337269),
 ('MOH', '2024-07-16', 0.08222401113669098),
 ('ZION', '2023-03-13', -0.2572490333713162),
 ('NEE', '20

In [16]:
# Create data frame of potential event dates

tickers = [item[0] for item in random_equityEvent_ls]
dates = [item[1] for item in random_equityEvent_ls]
returns = [item[2] for item in random_equityEvent_ls]

eventDate_df = pd.DataFrame({
    ('Potential Event Date', ''): dates,
    ('Return', ''): returns
}, index=tickers)

eventDate_df.index.name = 'Ticker'
eventDate_df.head()

,Potential Event Date,Return
,,
Ticker,,
WDC,2022-05-03,0.144659
DVA,2023-10-11,-0.168602
ALL,2022-10-20,-0.129042
ILMN,2022-01-11,0.169813
HRL,2023-10-12,-0.098261


In [17]:
# Download dataframe into Excel for event analysis in Bloomberg

# fh = '/Users/henrycosentino/Desktop/Python/Projects/Event Drift/Data/event_signal_random_set.xlsx'
# eventDate_df.to_excel(fh)

In [18]:
# Constructing a non-event data frame (this is a control variable group)

nonEvent_dict = id_event(main_df, equity_ls, sigma_lvl_equity=1, sigma_lvl_mrkt=20, key="_nonEvent")
nonEventCols_ls = list(nonEvent_dict.keys())

nonEventDummies_df = pd.DataFrame(nonEvent_dict, index=main_df.index)
nonEvent_df = pd.concat([main_df, nonEventDummies_df], axis=1)

truncated_nonEvent_df = nonEvent_df.iloc[:1000].copy() # Truncate to only capture event dates with a maximum of 60 trading days left in 2024

truncated_nonEvent_df.tail()

,PODD,CRWD,NWSA,SBAC,UPS,VZ,IPGP,PFG,NXPI,DE,...,AIG_nonEvent,BIO_nonEvent,HES_nonEvent,MSI_nonEvent,MDT_nonEvent,PYPL_nonEvent,WHR_nonEvent,GM_nonEvent,NI_nonEvent,LW_nonEvent
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2024-09-23,0.010102,-0.022378,0.005695,0.004426,0.001089,-0.001579,0.002562,0.001986,-0.000086,0.007539,...,1,1,1,1,1,1,1,0,1,1
2024-09-24,0.000167,-0.015692,0.008305,0.006028,0.006059,0.009038,0.010081,-0.005712,0.008152,0.004744,...,0,1,1,1,1,1,1,1,1,1
2024-09-25,-0.000879,-0.007174,-0.017222,-0.010951,0.015364,-0.002463,-0.010261,-0.004338,0.000941,-0.009638,...,1,0,0,1,1,1,1,0,1,1
2024-09-26,-0.014112,-0.017419,0.004952,-0.012828,0.013839,0.001571,0.058372,0.012600,0.035346,0.011157,...,1,0,1,0,1,0,0,1,1,0


In [19]:
# Constructing a non-event set to sample from

nonEvent_set = set()
for ticker in equity_ls:
    df = truncated_nonEvent_df[[ticker, f'{ticker}_nonEvent']]
    df = df[df[f'{ticker}_nonEvent'] == 1]
    
    for date, ret in zip(df[ticker].index, df[ticker]):
        if pd.isna(ret):
            continue
        elif isinstance(ret, str) and ret.lower() == 'nan':
            continue
        else:
            date = date.strftime('%Y-%m-%d')
            nonEvent_set.add((ticker, date, ret))

In [33]:
len(nonEvent_set)

253390

In [20]:
## NON-EVENTS ##

# Randomly sampling the equities with non-events

random_equityNonEvent_ls = random.sample(sorted(list(nonEvent_set)), 40) # Randomly sample n = 40
random_equityNonEvent_ls

[('PG', '2024-08-29', 0.005737590482213673),
 ('AOS', '2023-05-30', 0.0003001836417573234),
 ('ALL', '2023-11-27', 0.006295426385535485),
 ('NWL', '2023-07-27', -0.02007667986162631),
 ('CTVA', '2023-06-07', 0.008156969008952908),
 ('RVTY', '2024-08-14', -0.01368933988461807),
 ('ED', '2022-12-13', 0.001626951899488693),
 ('ARE', '2022-04-12', -0.005590410736999618),
 ('TSCO', '2022-02-16', -0.0203180465031354),
 ('CZR', '2022-03-15', 0.04168403430757506),
 ('TXT', '2022-03-29', 0.01712813465388807),
 ('BALL', '2023-06-21', -0.009107257238379463),
 ('GDDY', '2022-03-22', 0.01533366108285183),
 ('DVN', '2022-11-07', 0.007834261880459437),
 ('IEX', '2022-09-12', 0.005379959599924655),
 ('NEM', '2022-02-15', -0.01662775978454023),
 ('TFC', '2023-09-21', -0.007402304433144713),
 ('FOX', '2024-08-13', 0.0002757053710986668),
 ('CF', '2023-12-19', 0.01437831451669336),
 ('FRT', '2024-03-15', -0.002214572090931122),
 ('FI', '2024-03-28', 0.003075412587289383),
 ('CPT', '2023-12-12', 0.0072790

## Main Dataframe Construction (cont'd)

In [ ]:
# Reading in the data

fh = '/Users/henrycosentino/Desktop/Python/Projects/Event Drift/Data Processing/event_signal_random_set.xlsx'
main_event_df = pd.read_excel(fh)

main_event_df.head(5)

,ticker,potential_event_date,day_return,event_type_(good=1),event_name,event_summary,citation,Unnamed: 7
0,WDC,2022-05-03,0.144659,1.0,Activist Investor,Elliott Investment Management sent a letter to...,https://www.prnewswire.com/news-releases/ellio...,1
1,DVA,2023-10-11,-0.168602,0.0,Competitor Product,"Competitor, Novo Nordisk's Ozempic showed earl...",https://www.reuters.com/business/healthcare-ph...,1
2,ALL,2022-10-20,-0.129042,0.0,Negative Guidance,"Announced preliminary third quarter results, s...",https://blinks.bloomberg.com/news/stories/RK20...,1
3,ILMN,2022-01-11,0.169813,1.0,Investor Conference,The CEO and company presented strong guidance ...,https://investor.illumina.com/events-and-prese...,1
4,HRL,2023-10-12,-0.098261,0.0,Investor Day,The company presented results not in-line with...,https://investor.hormelfoods.com/news-and-even...,1


In [29]:
# Data frame clean-up

main_event_df.dropna(subset=['event_summary'], inplace=True)
main_event_df.drop(columns=['Unnamed: 7'], inplace=True)
main_event_df.reset_index(drop=True, inplace=True)
main_event_df.head(5)

,ticker,potential_event_date,day_return,event_type_(good=1),event_name,event_summary,citation
0,WDC,2022-05-03,0.144659,1.0,Activist Investor,Elliott Investment Management sent a letter to...,https://www.prnewswire.com/news-releases/ellio...
1,DVA,2023-10-11,-0.168602,0.0,Competitor Product,"Competitor, Novo Nordisk's Ozempic showed earl...",https://www.reuters.com/business/healthcare-ph...
2,ALL,2022-10-20,-0.129042,0.0,Negative Guidance,"Announced preliminary third quarter results, s...",https://blinks.bloomberg.com/news/stories/RK20...
3,ILMN,2022-01-11,0.169813,1.0,Investor Conference,The CEO and company presented strong guidance ...,https://investor.illumina.com/events-and-prese...
4,HRL,2023-10-12,-0.098261,0.0,Investor Day,The company presented results not in-line with...,https://investor.hormelfoods.com/news-and-even...


In [31]:
len(main_event_df)

40

In [23]:
# Return over horizon function

returns_df_c = returns_df.reset_index().copy()

def return_over_horizon(ret_df, main_df, stock, date, horizon=60):
    sub_ret_df = ret_df[['Date', 'SPY', stock]].copy()
    date_idx = sub_ret_df[sub_ret_df['Date'] == date].index

    start_index = date_idx[0] + 1 # Move one day forward to get the trading day after the event occurs...
    end_index = start_index + horizon

    # Stock return variable
    returns_slice = sub_ret_df[stock].iloc[start_index:end_index]
    day_returns = returns_slice + 1
    cumulative_return_stock = day_returns.prod() - 1

    # Market return variable
    returns_slice = sub_ret_df['SPY'].iloc[start_index:end_index]
    day_returns = returns_slice + 1
    cumulative_return_market = day_returns.prod() - 1

    # Earnings dummy variable 
    date_ls = list(sub_ret_df['Date'].iloc[start_index:end_index])
    start_date = date_ls[0]
    end_date = date_ls[-1]
    earnDate_dummy_ls = list(main_df[f'{stock}_earnDateDummy'].loc[start_date:end_date])
    earnDate_dummy = 0
    if 1 in earnDate_dummy_ls:
        earnDate_dummy += 1

    return cumulative_return_stock, cumulative_return_market, earnDate_dummy

In [24]:
## EVENTS ##

# Iterating over different return horizons, and calculating/aggregating the relevant data into a dictionary for empirical analysis

eventType_dummy_ls = list(main_event_df['event_type_(good=1)'])
ticker_ls, date_ls = list(main_event_df['ticker']), list(main_event_df['potential_event_date'])

final_event_data_dict = {}
horizons = range(1,61)
col_names = ['stock_return', 'market_return', 'earnDate_dummy', 'event_type_(good=1, no-event=NaN)']

for horizon in horizons:
    stock_return_ls = []
    market_return_ls = []
    earnDate_dummy_ls = []
    
    for ticker, date in zip(ticker_ls, date_ls):
        cum_stock_ret, cum_market_ret, earnDate_dummy = return_over_horizon(returns_df_c, main_df, ticker, date, horizon=horizon)
        stock_return_ls.append(cum_stock_ret)
        market_return_ls.append(cum_market_ret)
        earnDate_dummy_ls.append(earnDate_dummy)

    col_names_horizon = [str(horizon)+f"_{x}" for x in col_names]
    horizon_data_ls = [stock_return_ls, market_return_ls, earnDate_dummy_ls, eventType_dummy_ls]

    final_event_data_dict[horizon] = dict(zip(col_names_horizon, horizon_data_ls))

In [25]:
## NON-EVENTS ##

# Iterating over different return horizons, and calculating/aggregating the relevant data into a dictionary for empirical analysis

ticker_ls = [x[0] for x in random_equityNonEvent_ls]
date_ls = [x[1] for x in random_equityNonEvent_ls]
eventType_ls = [np.nan] * len(ticker_ls)

final_nonEvent_data_dict = {}
horizons = range(1,61)
col_names = ['stock_return', 'market_return', 'earnDate_dummy', 'event_type_(good=1, no-event=NaN)']

for horizon in horizons:
    stock_return_ls = []
    market_return_ls = []
    earnDate_dummy_ls = []
    
    for ticker, date in zip(ticker_ls, date_ls):
        cum_stock_ret, cum_market_ret, earnDate_dummy = return_over_horizon(returns_df_c, main_df, ticker, date, horizon=horizon)
        stock_return_ls.append(cum_stock_ret)
        market_return_ls.append(cum_market_ret)
        earnDate_dummy_ls.append(earnDate_dummy)

    col_names_horizon = [str(horizon)+f"_{x}" for x in col_names]
    horizon_data_ls = [stock_return_ls, market_return_ls, earnDate_dummy_ls, eventType_ls]

    final_nonEvent_data_dict[horizon] = dict(zip(col_names_horizon, horizon_data_ls))

In [26]:
# Final dictionary of observed data

master_dict = {"event_data": final_event_data_dict,
               "nonEvent_data": final_nonEvent_data_dict}

In [27]:
# Downloading the final dictionary to a csv file

import json

# fh = '/Users/henrycosentino/Desktop/Python/Projects/Event Drift/master_data.csv'

# with open(fh, 'w') as filepath:
#     json.dump(master_dict, filepath, indent=4)